In [98]:
#I had to create a path for pymongo because Jupyter was claiming it wasn't being found in the directory
import sys
sys.path.append("/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages")

import os
import json
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

In [99]:
#This function reads the file into a json file
def get_fantasy(folder_name):
    stats = {}
    
    for filename in os.listdir(folder_name):
        with open(os.path.join(folder_name, filename), 'r') as f: 
            content = f.read() # open in readonly mode
            content = re.sub('\$','',content)
            content_json = json.loads(content)
            filename = filename.replace('.json', '')
            content_json2 = {}
            #content_json2[filename] = content_json
            stats = content_json
    return stats

In [100]:
#This saves the file into a mongo database
def save_fantasy(stats, database_name, collection_name, host_name, host_port):  
    try:
        client = pymongo.MongoClient(host_name, host_port)
        print ("Connected successfully!!!")
    except e:
        print ("Could not connect to MongoDB: %s" % e )
    else:
        client.drop_database(database_name)

        # use database named usgs or create it if not there already
        stats_db = client[database_name]
            # create collection named earthquakes or create it if not there already
        stats_coll = stats_db[collection_name]
            # add all the earthquakes to the list
        stats_coll.insert_many(stats)
        print(f"Added {len(stats)} stats to collection {collection_name}")
        # close the database connection
        client.close()

In [101]:
#This function loads the mongodb file into a pandas dataframe for easy use
def load_stats(database_name, collection_name, host_name, host_port):  
    try:
        client = pymongo.MongoClient(host_name, host_port)
        print ("Connected successfully!!!")
    except e:
        print ("Could not connect to MongoDB: %s" % e )
    else:

        # use database named usgs or create it if not there already
        fantasy_db = client[database_name]
            # create collection named earthquakes or create it if not there already
        fantasy_coll = fantasy_db[collection_name]
        
        stats_list = {}
        for doc in fantasy_coll.find():
            ID = doc['_id']
            year = doc['Year']
            name = doc['name']
            position = doc['position']
            team = doc['team']
            adp = doc['adp']
            player_id = doc['PlayerID']
            age = doc['Age']
            att = doc['Att']
            cmp = doc['Cmp']
            FL = doc['FL']
            Fmb = doc['Fmb']
            G = doc['G']
            GS = doc['GS']
            Int = doc['Int']
            PosRk = doc['PosRk']
            PPR = doc['PPR']
            Rec = doc['Rec']
            RecTD = doc['RecTD']
            RecYds = doc['RecYds']
            Rk = doc['Rk']
            RushAtt = doc['RushAtt']
            RushTD = doc['RushTD']
            RushYds = doc['RushYds']
            TD = doc['TD']
            tgt = doc['Tgt']
            YA = doc['YA']
            Yds = doc['Yds']
            YR = doc['YR']
            stats_list[ID['oid']] = ([ID['oid'], year, name, position, team, adp, player_id, age, att, cmp, FL, Fmb, G, GS, Int, PosRk, PPR, Rec, RecTD, RecYds, Rk, RushAtt, RushTD, RushYds, TD, tgt, YA, Yds, YR])
        client.close()
        
        df = pd.DataFrame(stats_list.values(), columns=['_id', 'year', 'name', 'position', 'team', 'adp', 'PlayerID', 'Age', 'Att', 'Cmp', 'FL', 'Fmb', 'G', 'GS', 'Int', 'PosRk', 'PPR', 'Rec', 'RecTD', 'RecYds', 'Rk', 'RushAtt', 'RushTD', 'RushYds', 'TD', 'Tgt', 'YA', 'Yds', 'YR'])
        return df

In [102]:
folder_name = 'Fantasy'
database_name = 'fantasy_football'
collection_name = 'fantasy_data'
host_name = 'localhost'
host_port = 27017
stats = get_fantasy(folder_name)
save_fantasy(stats, database_name, collection_name, host_name, host_port)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [92]:
#This first function allows us to see the team that has the best average of players with the lowest ADP or average draft position
#This helps us indicate which team has the best success in terms of fantasy players
def team_adp_average(stats_list):
    team_group = stats_list.groupby(stats_list.team).mean().sort_values('adp',ascending=True)
    print(team_group)

#After seeing the best team is Cincinatti, this function allows us to see which of their players have had the best adp over the 7 years
def team_top_players(stats_list, team):
    players = stats_list[stats_list.team == team].sort_values('adp')
    print(players)
    
    
stats_data = load_stats(database_name, collection_name, host_name, host_port)
team_adp_average(stats_data)
team_top_players(stats_data, 'CIN')

Connected successfully!!!


KeyError: 'Age'